## Semantic Kernel with Fabric SQL Endpoint

- Leverage Semantic Kernel with your SQL Endpoint
- Would be great to include a tool with sample query statements
- Setup your .env file 
- Need **ODBC Driver 18 for SQL Server**


### Sample .env file:

```
FABRIC_CONNECTION_STRING=GETTHISFROMINFABRIC.datawarehouse.fabric.microsoft.com
FABRIC_WAREHOUSE=GETTHISFROMINFABRIC
FABRIC_TEST_TABLE=GETTHISFROMINFABRIC

GLOBAL_LLM_SERVICE="AzureOpenAI"
AZURE_OPENAI_API_KEY="PUTYOURKEYHERE"
AZURE_OPENAI_ENDPOINT="https://INFOUNDRY.openai.azure.com/"
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="gpt-4.1"
AZURE_OPENAI_TEXT_DEPLOYMENT_NAME="gpt-4.1"
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME="text-embedding-3-large"
AZURE_OPENAI_API_VERSION="2024-12-01-preview"
##Semantic Kernel Settings:
AZURE_AISEARCH_API_KEY=""
AZURE_AISEARCH_URL=""
```

In [7]:
# Note: if using a virtual environment, do not run this cell
#%pip install -U semantic-kernel
from semantic_kernel import __version__

__version__


'1.34.0'

In [ ]:
#from services import Service
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions import KernelArguments


from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

from semantic_kernel.contents import ChatMessageContent, TextContent, ImageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

import os
from Plugin.FabricSQLPlugin import FabricSQLPlugin


In [ ]:

kernel = Kernel()
service_id = "default"
kernel.remove_all_services()
kernel.add_service(AzureChatCompletion(service_id=service_id,),)
kernel.add_plugin(FabricSQLPlugin(),plugin_name="FabricSQL",)

KernelPlugin(name='FabricSQL', description=None, functions={'execute_sql': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='execute_sql', plugin_name='FabricSQL', description="\n                     With a SELECT statement using fully qualified name: database.dbo.table,  \n                     An Example query woudl be: 'SELECT TOP 10 PostalCode FROM [warehouse].[dbo].[dimension_customer];'\n                     query the database, and return the results as an HTML table\n                     ", parameters=[KernelParameterMetadata(name='warehouse', description=None, default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), KernelParameterMetadata(name='sql_query', description=None, default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=False, return_parameter=Kerne

In [10]:
system_message = """
You are a Fabric SQL expert.  You have tools to explore the database by getting tables, getting the schema and executing SQL select statements.
Explore the schema of tables to create sql to answer the questions asked.
You can ask clarifying questions if needed.

You have information regarding World Wide Importers Customer Profiles from a Fabric SQL Endpoint
To Write Valid SQL - you must use the Fully Qualifed Name [warehouse].[schema].[table] in ALL SQL queries.

You do have a tool to determine what the warehouse is, what tables are available, and what their schema is.
The correct schema will be required for any joins you do on the data to answer complex questions.
"""

In [11]:
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior


chat_completion : AzureChatCompletion = kernel.get_service(type=ChatCompletionClientBase)

# Enable planning
execution_settings = AzureChatPromptExecutionSettings(tool_choice="auto")
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
# Create a history of the conversation
history = ChatHistory()
history.add_system_message(system_message)
history.add_user_message("What is the schema of the table  'dimension_customer'")

result = (await chat_completion.get_chat_message_contents(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
        arguments=KernelArguments(),
    ))[0]
print(result)


history.add_assistant_message(str(result))


The schema of the 'dimension_customer' table is as follows:

- CustomerKey (int, nullable)
- WWICustomerID (int, nullable)
- Customer (varchar(8000), nullable)
- BillToCustomer (varchar(8000), nullable)
- Category (varchar(8000), nullable)
- BuyingGroup (varchar(8000), nullable)
- PrimaryContact (varchar(8000), nullable)
- PostalCode (varchar(8000), nullable)
- ValidFrom (datetime2, nullable)
- ValidTo (datetime2, nullable)
- LineageKey (int, nullable)

If you want information from any specific columns or a sample of the data, let me know!


In [12]:
history.add_user_message("great, what are 10 zip codes that are the most common for customers?")

result = (await chat_completion.get_chat_message_contents(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
        arguments=KernelArguments(),
    ))[0]
print(result)

wwilakehouse_01
SELECT TOP 10 PostalCode, COUNT(*) as CustomerCount
FROM [wwilakehouse_01].[dbo].[dimension_customer]
GROUP BY PostalCode
ORDER BY CustomerCount DESC;
Here are the 10 most common zip codes (PostalCodes) for customers and their counts:

1. 90683 - 5 customers
2. 90298 - 4 customers
3. 90761 - 3 customers
4. 90179 - 3 customers
5. 90588 - 3 customers
6. 90686 - 3 customers
7. 90706 - 3 customers
8. 90050 - 2 customers
9. 90644 - 2 customers
10. 90451 - 2 customers

Let me know if you want more details or a breakdown for other columns!
